# Initial setup for modelling

In [7]:
#!pip install pandas==1.0.3

In [2]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
from math import sqrt
import statsmodels.api as sm
from fbprophet import Prophet
from datetime import datetime, timedelta
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
import itertools
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

Importing plotly failed. Interactive plots will not work.


In [3]:
file = open('store_v2.p', 'rb')
store = pickle.load(file)
file.close()

In [4]:
store_ids = store.keys()

## Defining model kpis

In [5]:
# Calculate KPIs

def calc_kpis(actuals, predictions, seasonality=7):
    
    #Experemential
    try:
        actuals.reset_index(drop=True,inplace=True)
        predictions.reset_index(drop=True,inplace=True)
    except:
        pass
    
    mae_kpi = mean_absolute_error(actuals, predictions)
    
    rmse_kpi = sqrt(mean_squared_error(actuals, predictions))
    
    mase_kpi = (mean_absolute_error(actuals, predictions) / 
                mean_absolute_error(actuals[seasonality:], actuals[:-seasonality]))
    
    #smape_kpi = np.sum(np.absolute(actuals - predictions)/((np.absolute(actuals)+np.absolute(predictions))/2))*(100/len(predictions))
    
    smape_kpi=np.sum(np.absolute(actuals - predictions)/((np.absolute(actuals)+np.absolute(predictions))/2))*(100/len(predictions))
    
    return mae_kpi, rmse_kpi, mase_kpi, smape_kpi

In [6]:
def store_kpis(store_id,model_name,kpis):
    
    model_performance.loc[(store_id,model_name),["mae"]] = kpis[0] # "mae_kpi"
    model_performance.loc[(store_id,model_name),["rmse"]] = kpis[1] # rmse_kpi
    model_performance.loc[(store_id,model_name),["mase"]] = kpis[2] # mase_kpi
    model_performance.loc[(store_id,model_name),["smape"]] = kpis[3] # smape_kpi    

## Setup df to store model performances

In [7]:
models = ["bl_naive2", "sarima", "prophet"]
multi_index = [store_ids,models]
kpis = ["mae","rmse","mase","smape","rel_mase", "rel_smape", "owa","rank"]
model_performance = pd.DataFrame(index=pd.MultiIndex.from_product(multi_index, names=['store', 'model']), columns=kpis)

In [8]:
model_performance

mae rmse mase smape rel_mase rel_smape  owa rank
store model                                                      
1     bl_naive2  NaN  NaN  NaN   NaN      NaN       NaN  NaN  NaN
      sarima     NaN  NaN  NaN   NaN      NaN       NaN  NaN  NaN
      prophet    NaN  NaN  NaN   NaN      NaN       NaN  NaN  NaN
2     bl_naive2  NaN  NaN  NaN   NaN      NaN       NaN  NaN  NaN
      sarima     NaN  NaN  NaN   NaN      NaN       NaN  NaN  NaN
...              ...  ...  ...   ...      ...       ...  ...  ...
1114  sarima     NaN  NaN  NaN   NaN      NaN       NaN  NaN  NaN
      prophet    NaN  NaN  NaN   NaN      NaN       NaN  NaN  NaN
1115  bl_naive2  NaN  NaN  NaN   NaN      NaN       NaN  NaN  NaN
      sarima     NaN  NaN  NaN   NaN      NaN       NaN  NaN  NaN
      prophet    NaN  NaN  NaN   NaN      NaN       NaN  NaN  NaN

[3345 rows x 8 columns]

# Models

## Baseline Models

### SNAIVE2

In [9]:
# SNAIVE2 Baseline Model (S)ARIMA(X)(0,0,0)x(0,1,0)
def bl_naive2(data):
    
    data_train = data.loc[data["train_set"]==True,["Sales"]]
    data_test = data.loc[data["train_set"]==False,"Sales"]
    
    pred_start = data_train.index.max() + pd.DateOffset(1)
    pred_end = data_test.index.max()
    
    model = sm.tsa.statespace.SARIMAX(
        data_train,
        order=(0,0,0),
        seasonal_order=(0,1,0,7)
    ).fit()
    
    y_hat = model.get_prediction(
        start=pred_start,
        end=pred_end,
        dynamic=False
    )

    return dict(
        pred = y_hat.predicted_mean,
        kpis = calc_kpis(data_test, y_hat.predicted_mean, 7)
    )

In [10]:
def apply_bl_naive2(data):
    store_result = bl_naive2(store[store_id])
    store_kpis(store_id,"bl_naive2",store_result["kpis"])    

In [11]:
#TESTING
#bl_snaive2(store[13])

In [12]:
%%time
for store_id in store_ids:
    print("Applying NAIVE2 for store #"+str(store_id)+" out of "+ str(len(store_ids)))
    apply_bl_naive2(store[store_id])

Applying NAIVE2 for store #1 out of 1115
Applying NAIVE2 for store #2 out of 1115
Applying NAIVE2 for store #3 out of 1115
Applying NAIVE2 for store #4 out of 1115
Applying NAIVE2 for store #5 out of 1115
Applying NAIVE2 for store #6 out of 1115
Applying NAIVE2 for store #7 out of 1115
Applying NAIVE2 for store #8 out of 1115
Applying NAIVE2 for store #9 out of 1115
Applying NAIVE2 for store #10 out of 1115
Applying NAIVE2 for store #11 out of 1115
Applying NAIVE2 for store #12 out of 1115
Applying NAIVE2 for store #13 out of 1115
Applying NAIVE2 for store #14 out of 1115
Applying NAIVE2 for store #15 out of 1115
Applying NAIVE2 for store #16 out of 1115
Applying NAIVE2 for store #17 out of 1115
Applying NAIVE2 for store #18 out of 1115
Applying NAIVE2 for store #19 out of 1115
Applying NAIVE2 for store #20 out of 1115
Applying NAIVE2 for store #21 out of 1115
Applying NAIVE2 for store #22 out of 1115
Applying NAIVE2 for store #23 out of 1115
Applying NAIVE2 for store #24 out of 1115
A

Applying NAIVE2 for store #198 out of 1115
Applying NAIVE2 for store #199 out of 1115
Applying NAIVE2 for store #200 out of 1115
Applying NAIVE2 for store #201 out of 1115
Applying NAIVE2 for store #202 out of 1115
Applying NAIVE2 for store #203 out of 1115
Applying NAIVE2 for store #204 out of 1115
Applying NAIVE2 for store #205 out of 1115
Applying NAIVE2 for store #206 out of 1115
Applying NAIVE2 for store #207 out of 1115
Applying NAIVE2 for store #208 out of 1115
Applying NAIVE2 for store #209 out of 1115
Applying NAIVE2 for store #210 out of 1115
Applying NAIVE2 for store #211 out of 1115
Applying NAIVE2 for store #212 out of 1115
Applying NAIVE2 for store #213 out of 1115
Applying NAIVE2 for store #214 out of 1115
Applying NAIVE2 for store #215 out of 1115
Applying NAIVE2 for store #216 out of 1115
Applying NAIVE2 for store #217 out of 1115
Applying NAIVE2 for store #218 out of 1115
Applying NAIVE2 for store #219 out of 1115
Applying NAIVE2 for store #220 out of 1115
Applying NA

Applying NAIVE2 for store #390 out of 1115
Applying NAIVE2 for store #391 out of 1115
Applying NAIVE2 for store #392 out of 1115
Applying NAIVE2 for store #393 out of 1115
Applying NAIVE2 for store #394 out of 1115
Applying NAIVE2 for store #395 out of 1115
Applying NAIVE2 for store #396 out of 1115
Applying NAIVE2 for store #397 out of 1115
Applying NAIVE2 for store #398 out of 1115
Applying NAIVE2 for store #399 out of 1115
Applying NAIVE2 for store #400 out of 1115
Applying NAIVE2 for store #401 out of 1115
Applying NAIVE2 for store #402 out of 1115
Applying NAIVE2 for store #403 out of 1115
Applying NAIVE2 for store #404 out of 1115
Applying NAIVE2 for store #405 out of 1115
Applying NAIVE2 for store #406 out of 1115
Applying NAIVE2 for store #407 out of 1115
Applying NAIVE2 for store #408 out of 1115
Applying NAIVE2 for store #409 out of 1115
Applying NAIVE2 for store #410 out of 1115
Applying NAIVE2 for store #411 out of 1115
Applying NAIVE2 for store #412 out of 1115
Applying NA

Applying NAIVE2 for store #586 out of 1115
Applying NAIVE2 for store #587 out of 1115
Applying NAIVE2 for store #588 out of 1115
Applying NAIVE2 for store #589 out of 1115
Applying NAIVE2 for store #590 out of 1115
Applying NAIVE2 for store #591 out of 1115
Applying NAIVE2 for store #592 out of 1115
Applying NAIVE2 for store #593 out of 1115
Applying NAIVE2 for store #594 out of 1115
Applying NAIVE2 for store #595 out of 1115
Applying NAIVE2 for store #596 out of 1115
Applying NAIVE2 for store #597 out of 1115
Applying NAIVE2 for store #598 out of 1115
Applying NAIVE2 for store #599 out of 1115
Applying NAIVE2 for store #600 out of 1115
Applying NAIVE2 for store #601 out of 1115
Applying NAIVE2 for store #602 out of 1115
Applying NAIVE2 for store #603 out of 1115
Applying NAIVE2 for store #604 out of 1115
Applying NAIVE2 for store #605 out of 1115
Applying NAIVE2 for store #606 out of 1115
Applying NAIVE2 for store #607 out of 1115
Applying NAIVE2 for store #608 out of 1115
Applying NA

Applying NAIVE2 for store #780 out of 1115
Applying NAIVE2 for store #781 out of 1115
Applying NAIVE2 for store #782 out of 1115
Applying NAIVE2 for store #783 out of 1115
Applying NAIVE2 for store #784 out of 1115
Applying NAIVE2 for store #785 out of 1115
Applying NAIVE2 for store #786 out of 1115
Applying NAIVE2 for store #787 out of 1115
Applying NAIVE2 for store #788 out of 1115
Applying NAIVE2 for store #789 out of 1115
Applying NAIVE2 for store #790 out of 1115
Applying NAIVE2 for store #791 out of 1115
Applying NAIVE2 for store #792 out of 1115
Applying NAIVE2 for store #793 out of 1115
Applying NAIVE2 for store #794 out of 1115
Applying NAIVE2 for store #795 out of 1115
Applying NAIVE2 for store #796 out of 1115
Applying NAIVE2 for store #797 out of 1115
Applying NAIVE2 for store #798 out of 1115
Applying NAIVE2 for store #799 out of 1115
Applying NAIVE2 for store #800 out of 1115
Applying NAIVE2 for store #801 out of 1115
Applying NAIVE2 for store #802 out of 1115
Applying NA

Applying NAIVE2 for store #976 out of 1115
Applying NAIVE2 for store #977 out of 1115
Applying NAIVE2 for store #978 out of 1115
Applying NAIVE2 for store #979 out of 1115
Applying NAIVE2 for store #980 out of 1115
Applying NAIVE2 for store #981 out of 1115
Applying NAIVE2 for store #982 out of 1115
Applying NAIVE2 for store #983 out of 1115
Applying NAIVE2 for store #984 out of 1115
Applying NAIVE2 for store #985 out of 1115
Applying NAIVE2 for store #986 out of 1115
Applying NAIVE2 for store #987 out of 1115
Applying NAIVE2 for store #988 out of 1115
Applying NAIVE2 for store #989 out of 1115
Applying NAIVE2 for store #990 out of 1115
Applying NAIVE2 for store #991 out of 1115
Applying NAIVE2 for store #992 out of 1115
Applying NAIVE2 for store #993 out of 1115
Applying NAIVE2 for store #994 out of 1115
Applying NAIVE2 for store #995 out of 1115
Applying NAIVE2 for store #996 out of 1115
Applying NAIVE2 for store #997 out of 1115
Applying NAIVE2 for store #998 out of 1115
Applying NA

In [13]:
#model_performance

### SARIMA

In [14]:
# Step 1: Define possible combinations
# Use only 12 combinations
## ARIMA(2, d, 2)(1, D, 1)
## ARIMA(0, d, 0)(0, D, 0)
## ARIMA(1, d, 0)(1, D, 0)
## ARIMA(0, d, 1)(0, D, 1)

#[(2, d, 2), (1, D, 1, 7)],
#[(0, d, 0), (0, D, 0, 7)],
#[(1, d, 0), (1, D, 0, 7)],
#[(0, d, 1), (0, D, 1, 7)],
def create_model_cfgs():
    return(np.array([
    [(2, 0, 2), (1, 0, 1, 7)],
    [(2, 1, 2), (1, 1, 1, 7)],
    [(2, 0, 2), (1, 1, 1, 7)],
    [(2, 1, 2), (1, 0, 1, 7)],

    [(0, 0, 0), (0, 0, 0, 7)],
    [(0, 1, 0), (0, 1, 0, 7)],
    [(0, 0, 0), (0, 1, 0, 7)],
    [(0, 1, 0), (0, 0, 0, 7)],

    [(1, 0, 0), (1, 0, 0, 7)],
    [(0, 1, 1), (0, 1, 1, 7)],
    [(0, 0, 1), (0, 1, 1, 7)],
    [(0, 1, 1), (0, 0, 1, 7)]]))

In [15]:
# Step 2: Define SARIMA function to get AIC
def simulate_sarima(data, cfg):
    print("simulate "+str(cfg))
    order, sorder = cfg
    data_train = data.loc[data["train_set"]==True,["Sales"]]
    try:
        fit = sm.tsa.statespace.SARIMAX(data_train, order=order, seasonal_order=sorder).fit()
        return (cfg, fit.aic)
    except:
        return None

In [16]:
# Step 3: Run all possible combinations and return best configuration
def find_best_parameters(data):
    model_cfgs = create_model_cfgs()
    scores = list()
    for cfg in model_cfgs:
        scores.append(simulate_sarima(data, cfg))
        
    #executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
    #tasks = (delayed(simulate_sarima)(data, cfg) for cfg in model_cfgs)
    #scores = executor(tasks)
    
    # Step 4: Select best model cfg
    scores = [r for r in scores if r != None] #clean Nones
    scores.sort(key=lambda x: x[1])
    params, aic = scores[0]
    return params

In [17]:
# Step 5: Apply SARIMA with best parameters
def sarima(data,y="Sales"):
    
    print("Finding best parameters to forecast " + y)
    b_pdq, b_spdq = find_best_parameters(data) # find best parameters
    
    print("Best parameters have been found:")
    print(str(b_pdq) + str(b_spdq))
    
    data_train = data.loc[data["train_set"]==True,[y]]
    data_test = data.loc[data["train_set"]==False,y]
    
    pred_start = data_train.index.max() + pd.DateOffset(1)
    pred_end = data_test.index.max()
    
    print("Fitting model")
    model = sm.tsa.statespace.SARIMAX(
        data_train,
        order=b_pdq,
        seasonal_order=b_spdq
    ).fit()
    
    print("Forecasting future")
    y_hat = model.get_prediction(
        start=pred_start,
        end=pred_end,
        dynamic=False
    )
    print("DONE")
    return dict(
        pred = y_hat.predicted_mean,
        kpis = calc_kpis(data_test, y_hat.predicted_mean, 7)
    )    

In [18]:
def apply_sarima(data,y="Sales"):
    store_result = sarima(store[store_id],y)
    store_kpis(store_id,"sarima",store_result["kpis"])   

In [19]:
#TESTING
#%%time
#results = apply_sarima(store[13],"Sales")

In [23]:
%%time
#for store_id in store_ids:
for store_id in store_ids:
    print("Applying SARIMA for store #"+str(store_id)+" out of "+ str(len(store_ids)))
    apply_sarima(store[store_id])

Applying SARIMA for store #1 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 1, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #2 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 

simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 0, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #16 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 1, 2

simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 1, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #30 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 0, 2)(1, 1, 1, 7)
Fitting model
Foreca

simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 1, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #44 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 1, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA 

simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 1, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #58 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 0, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #59 out of 1115
Finding 

...

simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 1, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #1052 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 1, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #1053 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 

simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 0, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #1066 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 1, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #1067 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 

simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 0, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #1080 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 0, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #1081 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 

simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 1, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #1094 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 0, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #1095 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 

simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 0, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #1108 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 7)]
simulate [(0, 0, 0) (0, 1, 0, 7)]
simulate [(0, 1, 0) (0, 0, 0, 7)]
simulate [(1, 0, 0) (1, 0, 0, 7)]
simulate [(0, 1, 1) (0, 1, 1, 7)]
simulate [(0, 0, 1) (0, 1, 1, 7)]
simulate [(0, 1, 1) (0, 0, 1, 7)]
Best parameters have been found:
(2, 0, 2)(1, 1, 1, 7)
Fitting model
Forecasting future
DONE
Applying SARIMA for store #1109 out of 1115
Finding best parameters to forecast Sales
simulate [(2, 0, 2) (1, 0, 1, 7)]
simulate [(2, 1, 2) (1, 1, 1, 7)]
simulate [(2, 0, 2) (1, 1, 1, 7)]
simulate [(2, 1, 2) (1, 0, 1, 7)]
simulate [(0, 0, 0) (0, 0, 0, 7)]
simulate [(0, 1, 0) (0, 1, 0, 

In [21]:
model_performance

mae     rmse      mase    smape rel_mase rel_smape  owa  \
store model                                                                    
1     bl_naive2  1549.34  2280.79    1.3194  50.0204      NaN       NaN  NaN   
      sarima     678.244  1032.32  0.577586  46.6616      NaN       NaN  NaN   
      prophet        NaN      NaN       NaN      NaN      NaN       NaN  NaN   
2     bl_naive2  1693.23  2444.82   1.01657  48.7918      NaN       NaN  NaN   
      sarima     871.549  1304.88  0.523255  48.1454      NaN       NaN  NaN   
...                  ...      ...       ...      ...      ...       ...  ...   
1114  sarima         NaN      NaN       NaN      NaN      NaN       NaN  NaN   
      prophet        NaN      NaN       NaN      NaN      NaN       NaN  NaN   
1115  bl_naive2  2586.95  3865.92   1.13244  50.8799      NaN       NaN  NaN   
      sarima         NaN      NaN       NaN      NaN      NaN       NaN  NaN   
      prophet        NaN      NaN       NaN      NaN      NaN       NaN  NaN   

                rank  
store model           
1     bl_naive2  NaN  
      sarima     NaN  
      prophet    NaN  
2     bl_naive2  NaN  
      sarima     NaN  
...              ...  
1114  sarima     NaN  
      prophet    NaN  
1115  bl_naive2  NaN  
      sarima     NaN  
      prophet    NaN  

[3345 rows x 8 columns]

# Models with regressors

## FB Prophet

In [24]:
# Credits to https://www.kaggle.com/elenapetrova/time-series-analysis-and-forecasts-with-prophet
def get_holiday_df(df):
    state_dates = df[(df.StateHoliday == 'a') | (df.StateHoliday == 'b') & (df.StateHoliday == 'c')].loc[:, 'Date'].values
    school_dates = df[df.SchoolHoliday == 1].loc[:, 'Date'].values
    state = pd.DataFrame({'holiday': 'state_holiday',
                      'ds': pd.to_datetime(state_dates)})
    school = pd.DataFrame({'holiday': 'school_holiday',
                          'ds': pd.to_datetime(school_dates)})
    return pd.concat((state, school))

In [25]:
def fb_univariate(df, y, fb_holidays):
    
    fb_train = df.loc[df.train_set==True,["ds",y]]
    fb_train.columns = ("ds","y")
    
    m = Prophet(holidays = fb_holidays, weekly_seasonality=True)
    m.fit(fb_train)
    
    future = m.make_future_dataframe(periods=91,include_history = False)
    forecast = m.predict(future)
    return forecast["yhat"]

In [26]:
def fb_prophet(df, exog=None):
    
    # Splitting up train and test set
    fb_train = df.loc[df.train_set==True,]
    fb_test = df.loc[df.train_set==False,]
    
    # Get holidays
    fb_holidays = get_holiday_df(fb_train)
    
    # Rename for forecasting 
    fb_train.rename(columns={"Date": "ds", "Sales": "y"}, inplace=True)
    
    # Modeling
    m = Prophet(holidays = fb_holidays, weekly_seasonality=True)
    
    # Adding regressors
    if exog is not None:
        for ex in exog:
            m.add_regressor(ex)

        
    m.fit(fb_train)
    
    # Create future dataframe
    future = m.make_future_dataframe(periods=91,include_history = False) #+1 to get the 90 days
    
    # Predicting regressors first
    print("Predicting future values for exog regressors")
    if exog is not None:
        for ex in exog:
            future[ex] = fb_univariate(fb_train,ex,fb_holidays)
    print("--done")
    
    # Run forecast
    print("Start with forecasting")
    forecast = m.predict(future)
    print("--done")
    return dict(
        pred = forecast["yhat"],
        kpis = calc_kpis(fb_test["Sales"] , forecast["yhat"],7)
    )

In [27]:
def apply_fb_prophet(df,exog=None):
    store_result = fb_prophet(store[store_id],exog=None)
    store_kpis(store_id,"prophet",store_result["kpis"])    

In [28]:
%%time
#for store_id in [13]:
for store_id in store_ids:
    print("Applying Prophet for store #"+str(store_id)+" out of "+ str(len(store_ids)))
    apply_fb_prophet(store[store_id],["Customers","Promo"])

Applying Prophet for store #1 out of 1115


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Predicting future values for exog regressors
--done
Start with forecasting


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


--done
Applying Prophet for store #2 out of 1115
Predicting future values for exog regressors
--done
Start with forecasting


...

--done
Applying Prophet for store #1113 out of 1115
Predicting future values for exog regressors
--done
Start with forecasting


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


--done
Applying Prophet for store #1114 out of 1115
Predicting future values for exog regressors
--done
Start with forecasting


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


--done
Applying Prophet for store #1115 out of 1115
Predicting future values for exog regressors
--done
Start with forecasting
--done
Wall time: 33min 22s


In [ ]:
#model_performance

# Calculate OWA performance and ranks

In [29]:
# Store final model performance
#pickle.dump(model_performance, open("model_performance.p", "wb"))

In [44]:
# For backup purposes
file = open('model_performance.p', 'rb')
model_performance = pickle.load(file)
file.close()

In [45]:
model_performance

mae     rmse      mase    smape rel_mase rel_smape  owa  \
store model                                                                    
1     bl_naive2  1549.34  2280.79    1.3194  50.0204      NaN       NaN  NaN   
      sarima     678.244  1032.32  0.577586  46.6616      NaN       NaN  NaN   
      prophet    675.875  1052.31  0.575568  46.3825      NaN       NaN  NaN   
2     bl_naive2  1693.23  2444.82   1.01657  48.7918      NaN       NaN  NaN   
      sarima     871.549  1304.88  0.523255  48.1454      NaN       NaN  NaN   
...                  ...      ...       ...      ...      ...       ...  ...   
1114  sarima     3079.38  4601.92  0.689308  44.1977      NaN       NaN  NaN   
      prophet       2971  4587.55  0.665049  43.4402      NaN       NaN  NaN   
1115  bl_naive2  2586.95  3865.92   1.13244  50.8799      NaN       NaN  NaN   
      sarima     1399.49  2024.69  0.612633  51.2967      NaN       NaN  NaN   
      prophet    1418.11  1936.05  0.620782  49.3952      NaN       NaN  NaN   

                rank  
store model           
1     bl_naive2  NaN  
      sarima     NaN  
      prophet    NaN  
2     bl_naive2  NaN  
      sarima     NaN  
...              ...  
1114  sarima     NaN  
      prophet    NaN  
1115  bl_naive2  NaN  
      sarima     NaN  
      prophet    NaN  

[3345 rows x 8 columns]

In [46]:
for store_id in store_ids:
    for model in ["bl_naive2", "sarima", "prophet"]:
        
        model_performance.loc[(store_id, model),["rel_mase"]] = (
            model_performance.loc[(store_id, model),["mase"]].values/
            model_performance.loc[(store_id,"bl_naive2"),["mase"]].values
        )
    
        model_performance.loc[(store_id, model),["rel_smape"]] = (
            model_performance.loc[(store_id, model),["smape"]].values/
            model_performance.loc[(store_id,"bl_naive2"),["smape"]].values
        )
        
        model_performance.loc[(store_id, model),["owa"]] = (
            model_performance.loc[(store_id, model),["rel_mase"]].values+
            model_performance.loc[(store_id,"bl_naive2"),["rel_smape"]].values
        )/2 

In [48]:
for store_id in store_ids:
    model_performance.loc[(store_id,),["rank"]] = model_performance.loc[(1,),["owa"]].rank(method='min').values
    

In [49]:
model_performance

mae     rmse      mase    smape  rel_mase rel_smape  \
store model                                                                
1     bl_naive2  1549.34  2280.79    1.3194  50.0204         1         1   
      sarima     678.244  1032.32  0.577586  46.6616  0.437763  0.932852   
      prophet    675.875  1052.31  0.575568  46.3825  0.436234  0.927271   
2     bl_naive2  1693.23  2444.82   1.01657  48.7918         1         1   
      sarima     871.549  1304.88  0.523255  48.1454  0.514726  0.986753   
...                  ...      ...       ...      ...       ...       ...   
1114  sarima     3079.38  4601.92  0.689308  44.1977  0.522304   1.05993   
      prophet       2971  4587.55  0.665049  43.4402  0.503922   1.04177   
1115  bl_naive2  2586.95  3865.92   1.13244  50.8799         1         1   
      sarima     1399.49  2024.69  0.612633  51.2967  0.540984   1.00819   
      prophet    1418.11  1936.05  0.620782  49.3952   0.54818  0.970819   

                      owa rank  
store model                     
1     bl_naive2         1    3  
      sarima     0.718881    2  
      prophet    0.718117    1  
2     bl_naive2         1    3  
      sarima     0.757363    2  
...                   ...  ...  
1114  sarima     0.761152    2  
      prophet    0.751961    1  
1115  bl_naive2         1    3  
      sarima     0.770492    2  
      prophet     0.77409    1  

[3345 rows x 8 columns]

In [50]:
pickle.dump(model_performance, open("model_performance_done.p", "wb"))